# Курс Введение в обработку естественного языка

# Урок 8. Рекуррентные нейронные сети RNN LSTM GRU

## Задание
На вебинаре мы говорили что долгое время CNN и RNN архитектуры были конкурируещими. Выяснить, какая архитектура больше подходит для задачи сантимент анализа на данных с вебинара.
1. построить свёрточные архитектуры
2. построить различные архитектуры с RNN
3. построить совместные архитектуры CNN -> RNN и (RNN -> CNN)
4. сделать выводы что получилось лучше



## Решение

### Установка

In [4]:
!pip install pymorphy2
!pip install stop_words

     |████████████████████████████████| 55 kB 3.0 MB/s 
     |████████████████████████████████| 8.2 MB 12.9 MB/s 
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32911 sha256=ae2f2fee142a1291696d76a4c0b509abeb1e7e4c97573b61ccb028d435198f8a
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words


In [5]:
# Переключение версии TenzorFlow
%tensorflow_version 2.x

In [6]:
import warnings
warnings.filterwarnings("ignore")

from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from string import punctuation
import re

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalMaxPooling1D, Conv1D, Activation, Masking, SimpleRNN, Dropout, LSTM, GRU, Flatten, MaxPooling1D
# from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

### Загрузка данных

In [9]:
# Возьмем датасет с прошлого занятия

df_train = pd.read_csv("data/train.csv")
df_test = pd.read_csv("data/test.csv")
df_val = pd.read_csv("data/val.csv")

df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


### Предобработка

In [10]:
exclude = set(punctuation)
sw = set(get_stop_words("ru"))
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()

    txt = re.sub("\sне", "не", txt)

    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    # txt = [word for word in txt if len(word)>1] # условие "более одного слова в тексте"

    return " ".join(txt)

df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

### Сохранение результата предобработки

In [36]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [37]:
import pickle

with open('/content/drive/My Drive/Colab Notebooks/115_NLP/less_08/hw_08/prep/prep_df_train.pkl', 'wb+') as f:
    pickle.dump(df_train, f)
with open('/content/drive/My Drive/Colab Notebooks/115_NLP/less_08/hw_08/prep/prep_df_val.pkl', 'wb+') as f:
    pickle.dump(df_val, f)
with open('/content/drive/My Drive/Colab Notebooks/115_NLP/less_08/hw_08/prep/prep_df_test.pkl', 'wb+') as f:
    pickle.dump(df_test, f)

### Токенизация

In [11]:
text_corpus_train = df_train['text'].values
text_corpus_valid = df_val['text'].values
text_corpus_test = df_test['text'].values

In [12]:
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_val = tokenizer.texts_to_sequences(text_corpus_valid)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

x_train = pad_sequences(sequences_train, maxlen=training_length)
x_val = pad_sequences(sequences_val, maxlen=training_length)

In [13]:
y_train = df_train['class'].values
y_val = df_val['class'].values

### Накопление результатов

In [21]:
# накопление результатов
result = []

## 1.CNN

In [22]:
def model_CNN(word_count=258108, training_length=27, batch_size=512, epochs=10):
  model = tf.keras.Sequential()
  model.add(Embedding(input_dim=word_count, 
                      output_dim=128, 
                      input_length=training_length))
  model.add(Conv1D(128, 3, activation='relu'))
  model.add(GlobalMaxPooling1D())
  model.add(Dense(10, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))

  # компиляция модели
  model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
  
  model.summary()

  # обучение модели
  early_stopping=EarlyStopping(monitor='val_loss')  
  model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])
  
  # оценка качества модели
  [loss, accuracy] = model.evaluate(x_val, y_val)
  print("Test loss {:.4f}, accuracy {:.2f}%".format(loss, accuracy * 100))

  # накопление результатов
  result.append(['CNN', loss, accuracy])

  return result

In [23]:
model_CNN(word_count=word_count, training_length=training_length, batch_size=512, epochs=10)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 27, 128)           33037824  
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 25, 128)           49280     
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                1290      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 11        
Total params: 33,088,405
Trainable params: 33,088,405
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
709/709 [==============================] - 2s 2ms/step - loss: 0.5075 - accuracy: 0.7442
Test los

[['CNN', 0.5075072050094604, 0.7442137002944946]]

## 2.SimpleRNN

In [24]:
def model_simple_RNN(word_count=258108, training_length=27, batch_size=512, epochs=10):
  model = tf.keras.Sequential()

  model.add(
      Embedding(input_dim=word_count,
                input_length=training_length,
                output_dim=30,
                trainable=True,
                mask_zero=True))
  model.add(Masking(mask_value=0.0))

  model.add(SimpleRNN(64))
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(1, activation='sigmoid'))

  # компиляция модели
  model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
  
  model.summary()

  # обучение модели
  early_stopping=EarlyStopping(monitor='val_loss')  
  model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])
  
  # оценка качества модели
  [loss, accuracy] = model.evaluate(x_val, y_val)
  print("Test loss {:.4f}, accuracy {:.2f}%".format(loss, accuracy * 100))

  # накопление результатов
  result.append(['Simple_RNN', loss, accuracy])

  return result

In [25]:
model_simple_RNN(word_count=word_count, training_length=training_length, batch_size=512, epochs=10)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 27, 30)            7743240   
_________________________________________________________________
masking_1 (Masking)          (None, 27, 30)            0         
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 64)                6080      
_________________________________________________________________
dense_6 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 65        
Total params: 7,753,545
Trainable params: 7,753,545
Non-trainable params: 0
____________________________________________

[['CNN', 0.5075072050094604, 0.7442137002944946],
 ['Simple_RNN', 0.5063887238502502, 0.7432438135147095]]

## 3.LSTM с шириной слоя LSTM 128

In [26]:
def model_LSTM(word_count=258108, training_length=27, batch_size=512, epochs=10):
  model = tf.keras.Sequential()

  model.add(
      Embedding(input_dim=word_count,
                input_length=training_length,
                output_dim=30,
                trainable=True,
                mask_zero=True))
  model.add(Masking(mask_value=0.0))
  model.add(LSTM(128, recurrent_dropout=0.2))
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(1, activation='sigmoid'))

  # компиляция модели
  model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
  
  model.summary()

  # обучение модели
  early_stopping=EarlyStopping(monitor='val_loss')  
  model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])
  
  # оценка качества модели
  [loss, accuracy] = model.evaluate(x_val, y_val)
  print("Test loss {:.4f}, accuracy {:.2f}%".format(loss, accuracy * 100))

  # накопление результатов
  result.append(['LSTM', loss, accuracy])

  return result

In [27]:
model_LSTM(word_count=word_count, training_length=training_length, batch_size=512, epochs=10)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 27, 30)            7743240   
_________________________________________________________________
masking_2 (Masking)          (None, 27, 30)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               81408     
_________________________________________________________________
dense_8 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 65        
Total params: 7,832,969
Trainable params: 7,832,969
Non-trainable params: 0
____________________________________________

[['CNN', 0.5075072050094604, 0.7442137002944946],
 ['Simple_RNN', 0.5063887238502502, 0.7432438135147095],
 ['LSTM', 0.5083476305007935, 0.7447868585586548]]

## 4.GRU с шириной слоя GRU 128

In [28]:
def model_GRU(word_count=258108, training_length=27, batch_size=512, epochs=10):
  model = tf.keras.Sequential()

  model.add(
      Embedding(input_dim=word_count,
                input_length=training_length,
                output_dim=30,
                trainable=True,
                mask_zero=True))
  model.add(Masking(mask_value=0.0))
  model.add(GRU(128, recurrent_dropout=0.2))
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(1, activation='sigmoid'))

  # компиляция модели
  model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
  
  model.summary()

  # обучение модели
  early_stopping=EarlyStopping(monitor='val_loss')  
  model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])
  
  # оценка качества модели
  [loss, accuracy] = model.evaluate(x_val, y_val)
  print("Test loss {:.4f}, accuracy {:.2f}%".format(loss, accuracy * 100))

  # накопление результатов
  result.append(['GRU', loss, accuracy])

  return result

In [29]:
model_GRU(word_count=word_count, training_length=training_length, batch_size=512, epochs=10)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 27, 30)            7743240   
_________________________________________________________________
masking_3 (Masking)          (None, 27, 30)            0         
_________________________________________________________________
gru (GRU)                    (None, 128)               61440     
_________________________________________________________________
dense_10 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 65        
Total params: 7,813,001
Trainable params: 7,813,001
Non-trainable params: 0
____________________________________________

[['CNN', 0.5075072050094604, 0.7442137002944946],
 ['Simple_RNN', 0.5063887238502502, 0.7432438135147095],
 ['LSTM', 0.5083476305007935, 0.7447868585586548],
 ['GRU', 0.5268129706382751, 0.7363664507865906]]

## 5.CNN -> RNN

In [30]:
def model_CNN_RNN(word_count=258108, training_length=27, batch_size=512, epochs=10):
  model = tf.keras.Sequential()

  model.add(
      Embedding(input_dim=word_count,
                input_length=training_length,
                output_dim=30,
                trainable=True,
                mask_zero=True))
  model.add(Conv1D(64, 2, padding='same', activation='relu'))
  model.add(MaxPooling1D(2, padding='same'))
  model.add(Conv1D(32, 2, padding='same', activation='relu'))
  model.add(MaxPooling1D(2, padding='same'))


  model.add(LSTM(64, return_sequences=True, recurrent_dropout = 0.2))
  model.add(LSTM(32))
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(1, activation='sigmoid'))

  # компиляция модели
  model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
  
  model.summary()

  # обучение модели
  early_stopping=EarlyStopping(monitor='val_loss')  
  model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])
  
  # оценка качества модели
  [loss, accuracy] = model.evaluate(x_val, y_val)
  print("Test loss {:.4f}, accuracy {:.2f}%".format(loss, accuracy * 100))

  # накопление результатов
  result.append(['CNN_RNN', loss, accuracy])

  return result

In [31]:
model_CNN_RNN(word_count=word_count, training_length=training_length, batch_size=512, epochs=10)

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 27, 30)            7743240   
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 27, 64)            3904      
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 14, 64)            0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 14, 32)            4128      
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 7, 32)             0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 7, 64)             24832     
_________________________________________________________________
lstm_5 (LSTM)                (None, 32)               

[['CNN', 0.5075072050094604, 0.7442137002944946],
 ['Simple_RNN', 0.5063887238502502, 0.7432438135147095],
 ['LSTM', 0.5083476305007935, 0.7447868585586548],
 ['GRU', 0.5268129706382751, 0.7363664507865906],
 ['CNN_RNN', 0.5018177032470703, 0.7487105131149292]]

## 6.RNN -> CNN

In [32]:
def model_RNN_CNN(word_count=258108, training_length=27, batch_size=512, epochs=10):
  model = tf.keras.Sequential()

  model.add(
      Embedding(input_dim=word_count,
                input_length=training_length,
                output_dim=30,
                trainable=True,
                mask_zero=True))
  model.add(Masking(mask_value=0.0))
  model.add(LSTM(128, return_sequences=True, recurrent_dropout = 0.2))
  
  model.add(Conv1D(128, 2, padding='same', activation='relu'))
  model.add(MaxPooling1D(2, padding='same'))

  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(1, activation='sigmoid'))

  # компиляция модели
  model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
  
  model.summary()

  # обучение модели
  early_stopping=EarlyStopping(monitor='val_loss')  
  model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])
  
  # оценка качества модели
  [loss, accuracy] = model.evaluate(x_val, y_val)
  print("Test loss {:.4f}, accuracy {:.2f}%".format(loss, accuracy * 100))

  # накопление результатов
  result.append(['RNN_CNN', loss, accuracy])

  return result

In [33]:
model_RNN_CNN(word_count=word_count, training_length=training_length, batch_size=512, epochs=10)

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 27, 30)            7743240   
_________________________________________________________________
masking_4 (Masking)          (None, 27, 30)            0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 27, 128)           81408     
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 27, 128)           32896     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 14, 128)           0         
_________________________________________________________________
dense_14 (Dense)             (None, 14, 64)            8256      
_________________________________________________________________
dropout_6 (Dropout)          (None, 14, 64)           

[['CNN', 0.5075072050094604, 0.7442137002944946],
 ['Simple_RNN', 0.5063887238502502, 0.7432438135147095],
 ['LSTM', 0.5083476305007935, 0.7447868585586548],
 ['GRU', 0.5268129706382751, 0.7363664507865906],
 ['CNN_RNN', 0.5018177032470703, 0.7487105131149292],
 ['RNN_CNN', 0.6517332792282104, 0.5658928155899048]]

## Результат

In [35]:
pd.DataFrame(result, columns=['model', 'loss', 'accuracy'
                              ]).\
                              sort_values('loss',ascending=True).head(6)

,model,loss,accuracy
4,CNN_RNN,0.501818,0.748711
1,Simple_RNN,0.506389,0.743244
0,CNN,0.507507,0.744214
2,LSTM,0.508348,0.744787
3,GRU,0.526813,0.736366
5,RNN_CNN,0.651733,0.565893


Существенных различий в результатах применения рассмотренных архитектур в данном случае нет, за исключением совместной архитектуры 'RNN -> CNN', результат которой значительно отстает от остальных.